In [43]:
#從selenium引入webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains #協助定位
#引入beautifulsoup & 時間套件
from bs4 import BeautifulSoup
import time

import traceback
import os
import pymysql.cursors
from dotenv import load_dotenv
load_dotenv()

connection = pymysql.connect(host='localhost',
                             user='root',
                             password='admin1234',
                             db='fb_social_data',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

from datetime import date,datetime,timedelta
#today = str(date.today())
deadline = str(date.today() + timedelta(days=-365))
print(deadline)
from docx import Document #引入生成word檔套件

#初始化選項
options = Options()
options.add_argument("--disable-notifications")
#關掉圖片，可加快爬網頁速度
options.experimental_options['prefs'] = {'profile.default_content_settings':{'images':2},
                                        'profile.default_managed_default_content_settings':{'images':2}}
#背景執行chrome
driver = webdriver.Chrome(os.getcwd()+"/chromedriver", chrome_options=options)

try:
    with connection.cursor() as cursor:
        driver.get('https://www.facebook.com/')
        username = driver.find_elements_by_css_selector('input[name=email]')[0]
        password = driver.find_elements_by_css_selector('input[name=pass]')[0]
        username.send_keys(os.getenv('EMAIL'))
        password.send_keys(os.getenv('PASSWORD'))
        login_button = driver.find_elements_by_css_selector('input[type=submit]')[0]
        login_button.click()
        time.sleep(2)
        driver.get('https://www.facebook.com/toffee.miffy.mom/')
        for i in range(1,5):
            driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
            time.sleep(2)
        
        check = True
        while check:
            show_more_comments = driver.find_elements_by_css_selector('a._4sxc._42ft')
            for comment in show_more_comments:
                ActionChains(driver).move_to_element(comment).perform()
                comment.click()
                
            #print('open reply')
            time.sleep(2)
            #  點開後還有可能還有more
            if(len(show_more_comments) == 0):
                check = False
        
        sourceCode = BeautifulSoup(driver.page_source)
        article_box = sourceCode.select('div._1xnd')[1]
        articles = article_box.select('div._5pcr.userContentWrapper')
        for article in articles:
            utime = article.select('abbr._5ptz')[0]['data-utime']
            article_date = datetime.fromtimestamp(int(utime))
            check_date = article_date.strftime('%Y-%m-%d')
            #print(str(article_date))
            
            if(check_date > deadline):
                content = ''
                if(len(article.select('div.text_exposed_root')) == 0):
                    show_contents = article.select('div.userContent > p')
                    for show_content in show_contents:
                        content = content + show_content.text + '.'
                else:
                    show_contents = article.select('div.text_exposed_root > p')
                    hide_contents = article.select('div.text_exposed_show > p')
                    for show_content in show_contents:
                        content = content + show_content.text + '.'
                    for hide_content in hide_contents:
                        content = content + hide_content.text + '.'
                #print(content)

                if(len(article.select('tr._51mx')) > 0):
                    # 有 > 表示直屬於前面的物件，空格則是抓出所有物件
                    reatched = article.select('td._51m-.vMid.hLeft span')[0].text.replace(',','')
                    engagements = article.select('td._51mw._51m-.vMid.hLeft span')[0].text.replace(',','')
                #print('reatched:'+ reatched)
                #print('engagements:'+ engagements)

                like = article.select('span._81hb')[0].text.replace(',','')
                comment_elements = article.select('span._1whp._4vn2')
                share_elements = article.select('span._355t._4vn2')
                comment_num = share_num = 0
                if(len(comment_elements) > 0):
                    comment_num = comment_elements[0].text.split('Comment')[0].replace(',','')
                if(len(share_elements) > 0):
                    share_num = share_elements[0].text.split('Share')[0].replace(',','')
                #print('liked:'+ str(like) + 'comments:' + str(comment_num) + 'shared:'+ str(share_num))

                sql = '''
                SELECT date FROM `fb_social_data`.`posts` WHERE date = '{}'
                '''.format(article_date)
                cursor.execute(sql)
                posts = cursor.fetchall()

                if(len(posts) > 0):
                    sql = '''
                    UPDATE `fb_social_data`.`posts` 
                    SET date = '{}',
                    content = '{}',
                    reached = '{}',
                    engagements = '{}',
                    likes = '{}',
                    comments = '{}',
                    shared = '{}'
                    WHERE date = '{}'
                    '''.format(article_date,content,reatched,engagements,like,comment_num,share_num,article_date)
                else:              
                    sql = '''
                    INSERT INTO `fb_social_data`.`posts`(`date`,`content`,`reached`,`engagements`,`likes`,`comments`,`shared`) 
                    VALUES ('{}','{}','{}','{}','{}','{}','{}')
                    '''.format(article_date,content,reatched,engagements,like,comment_num,share_num)
                    #print(sql)
                cursor.execute(sql)
                connection.commit()

                sql = '''
                SELECT id FROM `fb_social_data`.`posts` WHERE date = '{}'
                '''.format(article_date)
                cursor.execute(sql)
                posts = cursor.fetchall()
                post_id = posts[0]['id']

                sql = '''
                DELETE FROM `fb_social_data`.`replies` WHERE post_id = '{}'
                '''.format(post_id)
                cursor.execute(sql)
                connection.commit()

                replied_comments = article.select('span._3l3x')
                for replied_comment in replied_comments:
                    replied_text = replied_comment.text
                    sql = '''
                    INSERT INTO `fb_social_data`.`replies`(`comment_text`,`post_id`) 
                    VALUES ('{}','{}')
                    '''.format(replied_text,post_id)
                    cursor.execute(sql)
                    connection.commit()
                    #print(replied_text)
        
    connection.close()
    driver.close()
except Exception:
    print(traceback.format_exc())
    connection.close()
    driver.close()

2019-04-28


<ipython-input-43-39c358f5a9fe>:35: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(os.getcwd()+"/chromedriver", chrome_options=options)


In [17]:
## 語意分析
import pymysql.cursors
import jieba
from snownlp import SnowNLP
import pprint
pp = pprint.PrettyPrinter(indent=4)

connection = pymysql.connect(host='localhost',
                             user='root',
                             password='admin1234',
                             db='fb_social_data',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

with connection.cursor() as cursor:
    sql = '''
    SELECT * FROM `fb_social_data`.`replies`'''
    cursor.execute(sql)
    replies = cursor.fetchall()
    text_dict = {}
    sentiments_dict = {}
    for reply in replies:
       # 斷詞 
        seg_list = jieba.cut(reply['comment_text'])       
        #print("|".join(seg_list))
       # 情緒分數
        s = SnowNLP(reply['comment_text'])
        sentiments_dict[reply['comment_text']] = s.sentiments
        
        
        for seg in seg_list:
            if seg in text_dict:
                text_dict[seg] += 1
            else:
                text_dict[seg] = 1
    
    text_data = sorted(text_dict.items() , key= lambda d:d[1] , reverse = True)
    pp.pprint(text_data)
    sentiments_data = sorted(sentiments_dict.items() , key= lambda d:d[1] , reverse = True)
    pp.pprint(sentiments_data)
    #print(sentiments_dict)
            
    
connection.close()

[   ('!', 5),
    ('筷子', 5),
    ('?', 5),
    ('~', 4),
    ('了', 4),
    ('家', 3),
    ('的', 3),
    (' ', 3),
    ('她', 3),
    ('你們', 2),
    ('要', 2),
    ('睡覺', 2),
    ('都', 2),
    ('"', 2),
    ('嗎', 2),
    ('很', 2),
    ('耶', 2),
    ('小', 2),
    ('覺得', 2),
    ('，', 2),
    ('會用', 2),
    ('有', 2),
    ('用', 2),
    ('～', 2),
    ('Miffy', 1),
    ('有點', 1),
    ('狂', 1),
    ('一定', 1),
    ('是', 1),
    ('教了', 1),
    ('‘', 1),
    ('’', 1),
    ('😆', 1),
    ('每天', 1),
    ('循循善誘', 1),
    ('親子館', 1),
    ('戲服', 1),
    ('周', 1),
    ('百合', 1),
    ('對', 1),
    ('啊', 1),
    ('！', 1),
    ('好好笑', 1),
    ('態度', 1),
    ('氣勢', 1),
    ('強', 1),
    ('👍', 1),
    ('也', 1),
    ('會', 1),
    ('驕傲', 1),
    ('女兒', 1),
    ('在', 1),
    ('雙妃', 1),
    ('。', 1),
    ('娘娘', 1),
    ('駕到', 1),
    ('本來', 1),
    ('就', 1),
    ('再', 1),
    ('但', 1),
    ('一', 1),
    ('學校', 1),
    ('午餐', 1),
    ('只有', 1),
    ('湯匙', 1),
    ('我們', 1),
    ('中班', 1),
    ('我', 1),
    ('厲害', 1